In [1]:
import sys
sys.path.append('../')

In [2]:
from agent import RandomAgent, TorchModelAgent
from simulator import YarnSimulator
from pathlib import Path
from tqdm.auto import trange
import pandas as pd
# import itertools
import mpire

In [3]:
n_runs = int(1e6)
seed = None

In [4]:
endings = []

def init_run(worker_state):
    # Create simulator
    worker_state['simulator'] = YarnSimulator(text_unk_macro="", yarn='../yarnScripts')
    # worker_state['endings'] = []
    pass

def run_game(worker_state, run):
    simulator = worker_state['simulator']
    simulator.restart()
    # simulator = YarnSimulator(text_unk_macro="", yarn='../yarnScripts')
    agent = RandomAgent(seed=seed)
    
    final = None
    # run simulator
    state, choices, extras = simulator.read()
    while not simulator.is_finished():
        # print(extras)
        # print(simulator.controller.state.attr)

        if (tmp := simulator.controller.state.attr.get('final', None)) is not None:
            final = (simulator.controller.state.title, tmp)

        choice = agent.act(state, choices)
        state, choices, extras = simulator.transition(choice)
    
    # worker_state['endings'] = worker_state['endings'].append(final)
    # worker_state['endings'].append(final)
    endings.append(final)
    return final

def exit_func(worker_state):
    # Return the list
    # return worker_state['endings']
    pass

with mpire.WorkerPool(n_jobs=None, use_worker_state=True, start_method="threading") as pool:
    pool.map(run_game, range(n_runs), worker_init=init_run)#, worker_exit=exit_func)
    # print(pool.get_exit_results())
    # endings = list(itertools.chain.from_iterable(pool.get_exit_results()))
    df = pd.DataFrame(data=endings, columns=['title', 'kind'])
    print(df)
    # df.write_excel('results.xlsx')


           title kind
0   MaloSilencio    0
1     MaloAmigos    0
2   MaloSilencio    0
3   MaloSilencio    0
4     MaloPadres    0
..           ...  ...
95  MaloSilencio    0
96  MaloSilencio    0
97    MaloAmigos    0
98    MaloAmanda    0
99  MaloSilencio    0

[100 rows x 2 columns]


## Random Agent

In [6]:
simulator = YarnSimulator(text_unk_macro="", yarn='../yarnScripts')

endings = []
for k in trange(int(n_runs)):
    # reset simulator and agent
    simulator.restart()
    agent = RandomAgent(seed=seed)
    # agent = TorchModelAgent(
    #     model_path=Path('../tmp/adamw_max_val_acc_8_False_125,[20],[20]_0.001'),
    #     rand=0,
    #     use_cpu=False,
    #     seed=seed,
    # )

    final = None
    # run simulator
    state, choices, extras = simulator.read()
    while not simulator.is_finished():
        # print(extras)
        # print(simulator.controller.state.attr)

        if (tmp := simulator.controller.state.attr.get('final', None)) is not None:
            final = (simulator.controller.state.title, tmp)

        choice = agent.act(state, choices)
        state, choices, extras = simulator.transition(choice)
    
    endings.append(final)

df = pd.DataFrame(data=endings, columns=['title', 'kind'])
        
print(df)


100%|██████████| 1000000/1000000 [3:24:11<00:00, 81.62it/s]    


               title kind
0       MaloSilencio    0
1       MaloSilencio    0
2         MaloPadres    0
3       MaloSilencio    0
4         MaloAmigos    0
...              ...  ...
999995  MaloSilencio    0
999996  MaloSilencio    0
999997  MaloSilencio    0
999998  MaloSilencio    0
999999  MaloSilencio    0

[1000000 rows x 2 columns]


AttributeError: 'DataFrame' object has no attribute 'write_excel'

In [12]:
df.to_csv('endings.csv')

In [10]:
df['kind'].value_counts(normalize=True)

0    0.968608
1    0.031392
Name: kind, dtype: float64

In [15]:
def init_func(worker_state):
    # Initialize a counter for each worker
    # worker_state['count_even'] = 0
    pass

def square_and_count_even(worker_state, x):
    # Count number of even numbers and return the square
    # if x % 2 == 0:
    #     worker_state['count_even'] += 1
    # return x * x
    pass

def exit_func(worker_state):
    # Return the counter
    # return worker_state['count_even']
    pass

with mpire.WorkerPool(n_jobs=4, use_worker_state=True, start_method="spawn") as pool:
    pool.map(square_and_count_even, range(100), worker_init=init_func, worker_exit=exit_func, progress_bar=True)
    print(pool.get_exit_results())  # Output, e.g.: [13, 13, 12, 12]
    print(sum(pool.get_exit_results()))  # Output: 50


Exception occurred, terminating ... :   0%|          | 0/100 [00:08<?, ?it/s]


KeyboardInterrupt: 

## Model Agent